In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model

In [6]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mediapipe as mp

In [3]:
# initialize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

In [4]:
# Load the gesture recognizer model
model = load_model('mp_hand_gesture')

In [5]:
# Load class names
f = open('gesture.names', 'r')
classNames = f.read().split('\n')
f.close()
print(classNames)

['Okay', 'Peace', 'Thumbs up', 'Thumbs down', 'Call me', 'Stop', 'Rock', 'Live long', 'Fist', 'Smile']


In [7]:
# Initialize the webcam
cap = cv2.VideoCapture(0)
while True:
    # Read each frame from the webcam
    _, frame = cap.read()

    x, y, c = frame.shape

    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get hand landmark prediction
    result = hands.process(framergb)

    # print(result)
    
    className = ''

    # post process the result
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                # print(id, lm)
                lmx = int(lm.x * x)
                lmy = int(lm.y * y)

                landmarks.append([lmx, lmy])


            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

            prediction = model.predict([landmarks])

            classID = np.argmax(prediction)
            className = classNames[classID]
    cv2.putText(frame, className, (15, 50), cv2.FONT_ITALIC, 
                   1.2, (255,0,100), 4, cv2.LINE_AA)


    cv2.imshow("WebCam1 (Press button 'B' for break)", frame) 

    if cv2.waitKey(1) == ord('b'):
        break

cap.release()
cv2.destroyAllWindows()